# Instacart Kaggle Project

In [3]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

In [8]:
#Read in Dataset
aisles_df=pd.read_csv(r"aisles.csv")
departments_df=pd.read_csv(r"departments.csv")
order_products_prior_df=pd.read_csv(r"order_products_prior.csv")
order_products_train_df=pd.read_csv(r"order_products_train.csv")
orders_df=pd.read_csv(r"orders.csv")
products_df=pd.read_csv(r"products.csv")

In [11]:
#Print data shapes
print("Aisles Dataset Shape : ",aisles_df.shape)
print("Departments Dataset Shape : ",departments_df.shape)
print("Order Products Prior Dataset Shape : ",order_products_prior_df.shape)
print("Order Products Train Dataset Shape : ",order_products_train_df.shape)
print("Orders Dataset Shape : ",orders_df.shape)
print("Products Dataset Shape : ",products_df.shape)

Aisles Dataset Shape :  (134, 2)
Departments Dataset Shape :  (21, 2)
Order Products Prior Dataset Shape :  (32434489, 4)
Order Products Train Dataset Shape :  (1384617, 4)
Orders Dataset Shape :  (3421083, 7)
Products Dataset Shape :  (49688, 4)


In [17]:
#Print data info
print("Aisles Dataset Info : ",aisles_df.info(),"\n")
print("Departments Dataset Info : ",departments_df.info(),"\n")
print("Order Products Prior Dataset Info : ",order_products_prior_df.info(),"\n")
print("Order Products Train Dataset Info : ",order_products_train_df.info(),"\n")
print("Orders Dataset Info : ",orders_df.info(),"\n")
print("Products Dataset Info : ",products_df.info(),"\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
aisle_id    134 non-null int64
aisle       134 non-null object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB
Aisles Dataset Info :  None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
department_id    21 non-null int64
department       21 non-null object
dtypes: int64(1), object(1)
memory usage: 416.0+ bytes
Departments Dataset Info :  None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(4)
memory usage: 989.8 MB
Order Products Prior Dataset Info :  None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 4 columns):
order_id             1384617 non-null int64
product_id           1384617